 # Module

In [1]:
from dotenv import load_dotenv
import os
from pydantic import BaseModel
import functools
import operator
import json
from IPython.display import Image, display

from typing import Annotated, Literal, Sequence
from typing_extensions import TypedDict

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, AIMessage, ToolMessage, HumanMessage
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_experimental.tools import PythonREPLTool
from langgraph.prebuilt import create_react_agent

# .env 파일 로드
load_dotenv()

True

# Create Tools

In [2]:
# search engine
tavily_tool = TavilySearchResults(max_results=5)

# code executer for chart
python_repl_tool = PythonREPLTool()

# Helper Utility
* Agent의 응답을 Human Message로 변환하는 역할

In [3]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {
        "messages": [HumanMessage(content=result["messages"][-1].content, name=name)]
    }

# Create Agent Supervisor
* 관리자 Agent는 필요에 따라 fuction calling을 출력할 것이며, 필요하지 않을 때는 finish를 출력하는 역할을 담당한다.

In [4]:
# 사용할 수 있는 fuctions
members = ["Researcher", "Coder"]

# supervisor의 system 프롬프트
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)

# Supervisor가 출력할 수 있는 options
options = ["FINISH"] + members

# Supervisor의 출력 스키마 정의
class routeResponse(BaseModel):
    next: Literal[*options]

# Supervisor의 전체 프롬프트
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

# LLM 정의
llm = ChatOpenAI(model="gpt-4o")

# Supervisor 노드 정의
def supervisor_agent(state):
    supervisor_chain = prompt | llm.with_structured_output(routeResponse)
    return supervisor_chain.invoke(state)

# Graph 설계

* State 정의

In [5]:
# State 정의
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str

* Research Node 정의

In [6]:
research_agent = create_react_agent(llm, tools=[tavily_tool])
research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

In [14]:
research_node.invoke("what is the capital of France")

AttributeError: 'functools.partial' object has no attribute 'invoke'

* Coder Node 정의

In [7]:
code_agent = create_react_agent(llm, tools=[python_repl_tool])
code_node = functools.partial(agent_node, agent=code_agent, name="Coder")

* Graph 정의 및 노드 추가

In [8]:
workflow = StateGraph(AgentState)
workflow.add_node("Researcher", research_node)
workflow.add_node("Coder", code_node)
workflow.add_node("supervisor", supervisor_agent)

* Graph Edge 추가

In [9]:
# 모든 도구들은 다시 supervisor로 return 한다
for member in members:
    workflow.add_edge(member, "supervisor")

# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)

# 시작은 supervisor로
workflow.add_edge(START, "supervisor")

# 그래프 빌드
graph = workflow.compile(
    checkpointer=MemorySaver()
)

# Run

In [10]:
config = {"configurable": {"thread_id": "1"}}

for s in graph.stream(
    {"messages": [HumanMessage(content="What is 3d segmentation? plz search it")]},
    {"recursion_limit": 50,
     "configurable": {"thread_id": "1"}},
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Researcher'}}
----
{'Researcher': {'messages': [HumanMessage(content='3D segmentation is a computer vision task that involves dividing a 3D point cloud or 3D mesh into semantically meaningful parts or regions. This process is essential in various applications, such as medical imaging, autonomous driving, and augmented reality, where understanding the structure and components of a 3D environment is crucial. Techniques for 3D segmentation often involve deep learning models that can accurately segment objects and features in 3D data. There are numerous research papers, benchmarks, and datasets available that focus on improving the efficiency and accuracy of 3D segmentation methods.', additional_kwargs={}, response_metadata={}, name='Researcher')]}}
----
{'supervisor': {'next': 'FINISH'}}
----


# Check

In [11]:
state_values = graph.get_state({"configurable": {"thread_id": "1"}}).values["messages"]

In [12]:
state_values

[HumanMessage(content='What is 3d segmentation? plz search it', additional_kwargs={}, response_metadata={}, id='caaab1b5-14f0-4507-bc48-a2354c4e9796'),
 HumanMessage(content='3D segmentation is a computer vision task that involves dividing a 3D point cloud or 3D mesh into semantically meaningful parts or regions. This process is essential in various applications, such as medical imaging, autonomous driving, and augmented reality, where understanding the structure and components of a 3D environment is crucial. Techniques for 3D segmentation often involve deep learning models that can accurately segment objects and features in 3D data. There are numerous research papers, benchmarks, and datasets available that focus on improving the efficiency and accuracy of 3D segmentation methods.', additional_kwargs={}, response_metadata={}, name='Researcher')]